<a href="https://colab.research.google.com/github/saparbayev-azizbek-12/bi-and-ai-talents-dl/blob/main/lesson-21/lesson_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = "/content/drive/MyDrive/dl_course/lesson-21/data/smile-annotations-final.csv"
df = pd.read_csv(file_path, header=None)

In [3]:
df.columns = ["id", "text", "label"]

In [4]:
df['label'].drop_duplicates()
rm_values = ["nocode", "not-relevant", "disgust|angry", "happy|surprise", "happy|sad", "sad|disgust", "sad|angry", "sad|disgust|angry"]
df = df[~df['label'].isin(rm_values)]
df['label'].value_counts()

,count
label,
happy,1137
angry,57
surprise,35
sad,32
disgust,6


In [5]:
labels = list(df["label"].drop_duplicates())
itos = {i:v for i,v in enumerate(labels)}
stoi = {v:i for i,v in itos.items()}
stoi

{'happy': 0, 'angry': 1, 'disgust': 2, 'sad': 3, 'surprise': 4}

In [6]:
X = df['text'].to_list()
Y = df["label"].apply(lambda row: stoi[row]).values

In [7]:
from sklearn.model_selection import train_test_split

Xtr, Xtemp, Ytr, Ytemp = train_test_split(
    X, Y,
    train_size=0.7,
    stratify=Y,
    shuffle=True,
    random_state=42
  )

Xval, Xts, Yval, Yts = train_test_split(
    Xtemp, Ytemp,
    train_size=0.5,
    stratify=Ytemp,
    shuffle=True,
    random_state=42
)


In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize(texts):
  return tokenizer(
      texts,
      padding=True,
      truncation=True,
      max_length=128,
      return_tensors='pt'
  )

Xtr_tokenized = tokenize(Xtr)
Xval_tokenized = tokenize(Xval)
Xts_tokenized = tokenize(Xts)

In [9]:
Xtr_tokenized['input_ids'][0]

tensor([  101,  1030,  8292, 15987,  2063,  1030, 12702, 19707,  3215,  1030,
         2329,  7606, 14820, 10166,  2307,  2000,  2156,  2009,   999,   999,
         3246,  2000,  2131,  2047, 10439,  2039,  2574,   999,  1025,  1007,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, X, Y):
    super().__init__()
    self.X = X
    self.Y = Y

  def __len__(self):
    return len(self.X['input_ids'])

  def __getitem__(self, id):
    message = {k: v[id] for k, v in self.X.items()}
    label = torch.tensor(self.Y[id])
    return message, label

Dtr = CustomDataset(Xtr_tokenized, Ytr)
Dval = CustomDataset(Xval_tokenized, Yval)
Dts = CustomDataset(Xts_tokenized, Yts)

In [11]:
Dtr[0]

({'input_ids': tensor([  101,  1030,  8292, 15987,  2063,  1030, 12702, 19707,  3215,  1030,
           2329,  7606, 14820, 10166,  2307,  2000,  2156,  2009,   999,   999,
           3246,  2000,  2131,  2047, 10439,  2039,  2574,   999,  1025,  1007,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 

In [22]:
DLtr = DataLoader(Dtr, batch_size=32, shuffle=True)
DLval = DataLoader(Dval, batch_size=32, shuffle=True)
DLts = DataLoader(Dts, batch_size=32, shuffle=False)

In [13]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=len(labels)
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [23]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=0.03)

In [16]:
def train_epoch(model, dataloader):
    model.train()
    total_loss = 0

    for inputs, labels in dataloader:

        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [17]:
def eval_epoch(model, dataloader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    acc = correct / total
    return total_loss / len(dataloader), acc


In [18]:
EPOCHS = 3

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, DLtr)
    val_loss, val_acc = eval_epoch(model, DLval)

    print(f"""
    Epoch {epoch+1}
    Train loss: {train_loss:.4f}
    Val loss:   {val_loss:.4f}
    Val acc:    {val_acc:.4f}
    """)



    Epoch 1
    Train loss: 0.7688
    Val loss:   0.4424
    Val acc:    0.9000
    

    Epoch 2
    Train loss: 0.3862
    Val loss:   0.2932
    Val acc:    0.9211
    

    Epoch 3
    Train loss: 0.2616
    Val loss:   0.2752
    Val acc:    0.9263
    
